*      OpenAI Agents SDK **

*      Context Management **

Context management

Context is an overloaded term. There are two main classes of context you might care about:

1. Context available locally to your code: this is data and dependencies you might need when tool 
   functions run, during callbacks like on_handoff, in lifecycle hooks, etc.

2. Context available to LLMs: this is data the LLM sees when generating a response.

Local context

This is represented via the RunContextWrapper class and the context property within it. The way this works is:

1. You create any Python object you want. A common pattern is to use a dataclass or a Pydantic object.

2. You pass that object to the various run methods (e.g. Runner.run(..., **context=whatever**)).

3. All your tool calls, lifecycle hooks etc will be passed a wrapper object, RunContextWrapper[T], where 
   T represents your context object type which you can access via wrapper.context.

The most important thing to be aware of: every agent, tool function, lifecycle etc for a given agent run must use the same type of context.

* You can use the context for things like:

* Contextual data for your run (e.g. things like a username/uid or other information about the user)

* Dependencies (e.g. logger objects, data fetchers, etc)

* Helper functions

سیاق و سباق کا انتظام

سیاق و سباق ایک اوور لوڈ شدہ اصطلاح ہے۔ سیاق و سباق کی دو اہم کلاسیں ہیں جن کی آپ کو پرواہ ہو سکتی ہے:

1. آپ کے کوڈ کے لیے مقامی طور پر دستیاب سیاق و سباق: یہ وہ ڈیٹا اور انحصار ہے جو آپ کو ٹول کے وقت درکار ہو سکتے ہیں۔ 
فنکشنز چلتے ہیں، کال بیکس کے دوران جیسے on_handoff، لائف سائیکل ہکس وغیرہ میں۔

2. LLMs کے لیے دستیاب سیاق و سباق: یہ وہ ڈیٹا ہے جسے LLM جواب پیدا کرتے وقت دیکھتا ہے۔

مقامی سیاق و سباق

اس کی نمائندگی RunContextWrapper کلاس اور اس کے اندر موجود سیاق و سباق کی خاصیت کے ذریعے کی جاتی ہے۔ اس کے کام کرنے کا طریقہ یہ ہے:

1. آپ کوئی بھی Python آبجیکٹ بناتے ہیں جو آپ چاہتے ہیں۔ ایک عام نمونہ ڈیٹا کلاس یا پیڈینٹک آبجیکٹ کا استعمال کرنا ہے۔

2. آپ اس اعتراض کو رن کے مختلف طریقوں پر منتقل کرتے ہیں (جیسے Runner.run(..., **context=whatever**))۔

3. آپ کی تمام ٹول کالز، لائف سائیکل ہکس وغیرہ کو ایک ریپر آبجیکٹ، RunContextWrapper[T] پاس کیا جائے گا، جہاں 
T آپ کے سیاق و سباق کے آبجیکٹ کی قسم کی نمائندگی کرتا ہے جس تک آپ wrapper.context کے ذریعے رسائی حاصل کر سکتے ہیں۔

سب سے اہم چیز جس کے بارے میں آگاہ ہونا ضروری ہے: ہر ایجنٹ، ٹول فنکشن، لائف سائیکل وغیرہ کو کسی مخصوص ایجنٹ کے لیے ایک ہی قسم کا سیاق و سباق استعمال کرنا چاہیے۔

* آپ سیاق و سباق کو چیزوں کے لیے استعمال کر سکتے ہیں جیسے:

* آپ کی دوڑ کے لیے متعلقہ ڈیٹا (مثلاً صارف کا نام/uid یا صارف کے بارے میں دیگر معلومات)

* انحصار (مثلاً لاگر آبجیکٹ، ڈیٹا فیچرز، وغیرہ)

* مددگار افعال

In [ ]:
import asyncio
from dataclasses import dataclass

from agents import Agent, RunContextWrapper, Runner, function_tool

@dataclass
class UserInfo:  
    name: str
    uid: int

@function_tool
async def fetch_user_age(wrapper: RunContextWrapper[UserInfo]) -> str:  
    """Fetch the age of the user. Call this function to get user's age information."""
    return f"The user {wrapper.context.name} is 47 years old"

async def main():
    user_info = UserInfo(name="John", uid=123)

    agent = Agent[UserInfo](  
        name="Assistant",
        tools=[fetch_user_age],
    )

    result = await Runner.run(  
        starting_agent=agent,
        input="What is the age of the user?",
        context=user_info,
    )

    print(result.final_output)  
    # The user John is 47 years old.

if __name__ == "__main__":
    asyncio.run(main())

Agent/LLM context

When an LLM is called, the only data it can see is from the conversation history. This means that if you want to make some new data available to the LLM, you must do it in a way that makes it available in that history. There are a few ways to do this:

1. You can add it to the Agent instructions. This is also known as a "system prompt" or "developer message". System prompts can be static strings, or they can be dynamic functions that receive the context and output a string. This is a common tactic for information that is always useful (for example, the user's name or the current date).

2. Add it to the input when calling the Runner.run functions. This is similar to the instructions 
   tactic, but allows you to have messages that are lower in the chain of command.
3. Expose it via function tools. This is useful for on-demand context - the LLM decides when it needs    
   some data, and can call the tool to fetch that data.

4. Use retrieval or web search. These are special tools that are able to fetch relevant data from files 
   or databases (retrieval), or from the web (web search). This is useful for "grounding" the response in relevant contextual data.


ایجنٹ/ایل ایل ایم سیاق و سباق

جب LLM کو کال کیا جاتا ہے، تو وہ صرف وہی ڈیٹا دیکھ سکتا ہے جو بات چیت کی سرگزشت سے ہوتا ہے۔ اس کا مطلب یہ ہے کہ اگر آپ LLM کے لیے کچھ نیا ڈیٹا دستیاب کرانا چاہتے ہیں، تو آپ کو اسے اس طرح کرنا چاہیے جس سے اسے اس تاریخ میں دستیاب ہو۔ ایسا کرنے کے چند طریقے ہیں:

1. آپ اسے ایجنٹ کی ہدایات میں شامل کر سکتے ہیں۔ اسے "سسٹم پرامپٹ" یا "ڈیولپر میسج" کے نام سے بھی جانا جاتا ہے۔ سسٹم پرامپٹس جامد تار ہو سکتے ہیں، یا وہ متحرک فنکشن ہو سکتے ہیں جو سیاق و سباق کو حاصل کرتے ہیں اور سٹرنگ آؤٹ پٹ کرتے ہیں۔ یہ معلومات کے لیے ایک عام حربہ ہے جو ہمیشہ مفید ہوتی ہے (مثال کے طور پر، صارف کا نام یا موجودہ تاریخ)۔

2. Runner.run فنکشنز کو کال کرتے وقت اسے ان پٹ میں شامل کریں۔ یہ ہدایات کی طرح ہے۔ 
حکمت عملی، لیکن آپ کو پیغامات حاصل کرنے کی اجازت دیتا ہے جو کمانڈ کے سلسلے میں کم ہیں۔
3. اسے فنکشن ٹولز کے ذریعے بے نقاب کریں۔ یہ آن ڈیمانڈ سیاق و سباق کے لیے مفید ہے - LLM فیصلہ کرتا ہے جب اسے ضرورت ہو۔ 
کچھ ڈیٹا، اور اس ڈیٹا کو لانے کے لیے ٹول کو کال کر سکتے ہیں۔

4. بازیافت یا ویب تلاش کا استعمال کریں۔ یہ خاص ٹولز ہیں جو فائلوں سے متعلقہ ڈیٹا حاصل کرنے کے قابل ہیں۔ 
یا ڈیٹا بیس (بازیافت)، یا ویب سے (ویب تلاش)۔ یہ متعلقہ سیاق و سباق کے اعداد و شمار میں ردعمل کو "گراؤنڈ" کرنے کے لیے مفید ہے۔